In [123]:
import pandas as pd
import pandas, sys
import numpy as np
import re
import sklearn
import spacy
import nltk
from nltk import sent_tokenize, word_tokenize
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Carmen\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Concatenating Data

In [210]:
df1 = pd.read_csv("../CSV_files/lang_doc.csv")
df2 = pd.read_csv("../CSV_files/lang_doc2.csv")
df3 = pd.read_csv("../CSV_files/lang_doc3.csv")
df4 = pd.read_csv("../CSV_files/lang_doc4.csv")
df5 = pd.read_csv("../CSV_files/lang_doc5.csv")
df6 = pd.read_csv("../CSV_files/lang_doc6.csv")
df7 = pd.read_csv("../CSV_files/lang_doc7.csv")
df8 = pd.read_csv("../CSV_files/lang_doc8.csv")
df9 = pd.read_csv("../CSV_files/lang_doc9.csv")
df10 = pd.read_csv("../CSV_files/lang_doc10.csv")
df11 = pd.read_csv("../CSV_files/lang_doc11.csv")
df12 = pd.read_csv("../CSV_files/lang_doc12.csv")

lang_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12],  ignore_index=True)

In [211]:
lang_df.head()

,Spanish_text,English_text
0,Esta tesis quiere profundizar sobre el caso de...,This thesis wants to delve into the case of th...
1,La violencia en sus diversas variantes está pr...,Violence in its several variations is present ...
2,"Esta tesis doctoral es un ensayo clínico que, ...",This doctoral thesis is a clinical trial that ...
3,La frustración es un fenómeno que puede mitiga...,Frustration can be mitigated if there has been...
4,Las políticas lingüísticas europeas promueven ...,European language policies promote the trainin...


In [212]:
lang_df.shape

(402, 2)

### More Cleaning + tokenizing sentences

In [213]:
lang_df[['Spanish_text', 'English_text']] = lang_df[['Spanish_text', 'English_text']].replace(r"\(.*\)","")

In [214]:
# remove any text that contains 'Español' in English_text variable
lang_df['English_text'] = lang_df['English_text'].apply(lambda x: 0 if 'Español' in x else x)

# remove rows with 0
lang_df = lang_df[lang_df['English_text'] != 0]

In [215]:
# storing each sentence in a list using sent_tokenize()
lang_df["tokenized_spanish"] = lang_df["Spanish_text"].apply(sent_tokenize)
lang_df["tokenized_english"] = lang_df["English_text"].apply(sent_tokenize)

In [216]:
lang_df.head()

,Spanish_text,English_text,tokenized_spanish,tokenized_english
0,Esta tesis quiere profundizar sobre el caso de...,This thesis wants to delve into the case of th...,[Esta tesis quiere profundizar sobre el caso d...,[This thesis wants to delve into the case of t...
1,La violencia en sus diversas variantes está pr...,Violence in its several variations is present ...,[La violencia en sus diversas variantes está p...,[Violence in its several variations is present...
2,"Esta tesis doctoral es un ensayo clínico que, ...",This doctoral thesis is a clinical trial that ...,"[Esta tesis doctoral es un ensayo clínico que,...",[This doctoral thesis is a clinical trial that...
3,La frustración es un fenómeno que puede mitiga...,Frustration can be mitigated if there has been...,[La frustración es un fenómeno que puede mitig...,[Frustration can be mitigated if there has bee...
4,Las políticas lingüísticas europeas promueven ...,European language policies promote the trainin...,[Las políticas lingüísticas europeas promueven...,[European language policies promote the traini...


In [217]:
# create variables with len in spanish and english
lang_df['len_token_spanish'] = lang_df['tokenized_spanish'].apply(lambda x: len(x))
lang_df['len_token_english'] = lang_df['tokenized_english'].apply(lambda x: len(x))

In [218]:
# dropped unequal sentence lengths
lang_df = lang_df[lang_df['len_token_spanish'] == lang_df['len_token_english']]

In [219]:
# drop irrelevent variables
lang_df.drop(['Spanish_text', 'English_text', 'len_token_spanish', 'len_token_english'], axis=1, inplace=True)

In [220]:
lang_df.head()

,tokenized_spanish,tokenized_english
1,[La violencia en sus diversas variantes está p...,[Violence in its several variations is present...
2,"[Esta tesis doctoral es un ensayo clínico que,...",[This doctoral thesis is a clinical trial that...
5,[Los pabellones de conveniencia han tenido un ...,[Flags of convenience have had a deep impact o...
9,[Los seres humanos y los animales experimentan...,[Humans and animals undergo morphological deve...
12,[La irrupción de los sistemas de derechos huma...,[The irruption of human rights systems at a co...


### 'exploding' the lists of sentences into own rows

In [221]:
# make sentences in the lists it's own row
lang_df2 =lang_df.apply(pd.Series.explode).reset_index()
lang_df2.drop('index', axis=1, inplace=True)

# renaming variables
lang_df2.rename(columns = {'tokenized_spanish' : 'Spanish', 'tokenized_english' : 'English'}, inplace = True)

In [222]:
len(lang_df2)

1888

### Example Sentences

In [223]:
lang_df2['Spanish'][600]

'Adicionalmente, con esta tesis se pretenden desarrollar técnicas efi-cientes que permitan su implementación en una red inalámbrica de senso-res acústicos en la cual el uso de otros métodos puede que no sea viable debido a restricciones tales como una capacidad de computo limitada o un ancho de banda restringido.Las propuestas de esta tesis vienen dadas por la necesidad de encontrar nuevas soluciones para el análisis de fuentes acústicas impulsivas dado cuenta con aplicaciones prácticas en múltiples campos tales como: vigi-lancia y seguridad, control de armas, análisis forense o táctica mili-tar, por nombrar unos pocos.'

In [224]:
lang_df2['English'][600]

'Thus, throughout the whole thesis, there is a constant tradeoff between performance and computational complexity.The proposals in this thesis come from the necessity for better automated impulsive sounds analysis systems given that they have practical applications in many fields such as forensics, surveillance and security, gun control or military tactics, to name a few.'

In [225]:
lang_df2.to_csv('../CSV_files/sent_translations.csv')